In [10]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
import sys 
import math
import operator
import re
import string
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('data/')
    train_file = base_path + 'QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    result = []
    for word in text:
        if word not in stopwords and word not in punctuations:
            result.append(word)
    return result
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        #RightAnser
        right_answer = []
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            answer = qa['answer_sentence']
            right_answer.append(answer)
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))
        #
        answer_id.append(right_answer)
        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
punctuations = [',','\'\'','?','\'','.','%','(',')',';','``']
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id_list= get_Docfrequency_SentenceBOW(dev)

In [11]:
'''
NER
'''
time_word = [
    'one','two','three','four','five','six','seven','eight','nine',
    'January','February','March','April','May','June','July','August','September','October','November','December',
    'million','billion',
    'minutes','hours','years','times',
    'mm','miles','inches','foot','feet','nm','cm','km','m',
    'late','early','around','over',
    'persons','seasons','square',
    'spring','summer','fall','autumn','winter'
]

location_word = [
    'southwest','southeast','northwest','northeast'
]

conjunction_word = ['and','of']

In [12]:
def input_NER():
    stanford_dir = os.path.join('stanford-ner-2016-10-31')
    jarfile = os.path.join(stanford_dir,'stanford-ner.jar')
    modelfile = os.path.join(stanford_dir,'classifiers/english.all.3class.distsim.crf.ser.gz')
    return modelfile,jarfile
model,jar = input_NER()
st = StanfordNERTagger(model,jar)

def analyse_NER(ner_sentences):
    result_sentences = []
    for ner_sentence in ner_sentences:
        result_sentence = []
        perv_type = u'O'
        word = u''
        conjunction = u''
        conjunc_flag = False
        for index,(entity,etype) in enumerate(ner_sentence):
            if perv_type == u'O' and etype != u'O':              
                perv_type = etype
                word = entity + u' '
            elif word != u'':
                if etype == u'O':
                    if entity not in conjunction_word:
                        result_sentence.append((word[:-1],perv_type))
                        word = u''
                        perv_type = u'O'
                        if conjunction != u'':
                            conjunction = u''
                            conjunc_flag = False
                    else:
                        if conjunction != u'':
                            conjunction = u''
                            conjunc_flag = False
                        else:
                            conjunction = entity
                            conjunc_flag = True
                elif etype != perv_type:
                    result_sentence.append((word[:-1],perv_type))
                    word = entity + u' '
                    perv_type = etype
                    conjunction = u''
                    conjunc_flag = False
                elif etype == perv_type:
                    if conjunc_flag:
                        if conjunction == u',':
                            word = word[:-1] + conjunction + u' ' + entity + u' '
                        else:
                            word = word + conjunction + u' ' + entity + u' '
                        conjunction = u''
                        conjunc_flag = False
                    else:
                        if entity in ['%'] or word == u'$ ':
                            word = word[:-1] + entity + u' '
                        else:
                            word = word + entity + u' '
        if word != u'':
            result_sentence.append((word[:-1],perv_type))
        result_sentences.append(result_sentence)      
    return result_sentences

def parse_NER(ner_sentences):
    pattern_number = re.compile(r'([0-9]+|\%|\$)')
    year_number = re.compile(r'([0-9]{4}s?)')
    result_sentences = []
    for ner_sentence in ner_sentences:
        result_sentence = []
        for index,(entity,etype) in enumerate(ner_sentence):
            if entity != u'':
                entity.replace(u'\u2013',u'-')
                entity.replace(u'\u2014',u'-')
                entity.replace(u'\u2212',u'-')
                entity.replace(u'\u2044',u'%')
                if etype == u'O':
                    if year_number.search(entity):
                        result_sentence.append((entity,u'YEAR'))
                    elif pattern_number.search(entity) or entity in time_word:
                        result_sentence.append((entity,u'NUMBER'))
                    elif u'-' in entity:
                        word_seperate = entity.split(u'-')
                        for word in word_seperate:
                            if word in time_word:
                                result_sentence.append((entity,u'NUMBER'))
                                break
                    elif entity in location_word:
                        result_sentence.append((entity,u'LOCATION'))
                    elif index == 0 and entity.lower() not in stopwords:
                        result_sentence.append((entity,u'ORGANIZATION'))
                    elif index != 0 and entity[0].isupper():
                        result_sentence.append((entity,u'ORGANIZATION'))
                    else:
                        result_sentence.append((entity,etype))
                elif entity in ['(',')']:
                    result_sentence.append((entity,u'O'))
                else:
                    result_sentence.append((entity,etype))
        result_sentences.append(result_sentence)
        
    return result_sentences

def extract_NER(parse_ner_sentence,mode):
    result = []
    if mode == 0:
        #PERSON
        for entity,etype in parse_ner_sentence:
            if etype == u'PERSON':
                result.append(entity)
    elif mode == 1:
        #NUMBER
        for entity,etype in parse_ner_sentence:
            if etype == u'NUMBER':
                result.append(entity)
    elif mode == 2:
        #LOCATION
        for entity,etype in parse_ner_sentence:
            if etype == u'LOCATION':
                result.append(entity)
    elif mode == 3:
        #ORGANIZATION
        for entity,etype in parse_ner_sentence:
            if etype == u'ORGANIZATION':
                result.append(entity)
    elif mode == 4:
        #YEAR
        for entity,etype in parse_ner_sentence:
            if etype == u'YEAR':
                result.append(entity)
    return result

def parse_token(token_sentence):
    result = []
    for index,word in enumerate(token_sentence):
        if index != 0 and index != (len(token_sentence)-1) and word == u'.':
            last_word = result[-1]
            last_word = last_word + u'.'
            result = result[:-1]
            result.append(last_word)
        else:
            result.append(word)
    return result

In [13]:
#Only consider queries with 'What'/'what'
def getHeadWord(text):
    text = text.encode('ascii','replace')
    pos =  pos_tag(word_tokenize(text))
    if pos[0]==('Which', 'JJ'):
        pos[0] = ('Which', 'WHICH')
    for i,item in enumerate(pos):   
        #print ('item',item)
        word = item[0].lower()
        ppos = item[1]
        if word=='what' and (ppos=='WP' or ppos=='WDT'):
            pos[i] = ('what', 'WHAT')   
        elif word=='which' and ppos=='JJ':
            pos[i] = ('which', 'WHICH')  
    #print pos
    grammar = r"""
                V: {<V.*>}          # Verb
                HEAD:
                    {<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?<NN.*>+}
                    {<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?<VBG.*>+}
                    }<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?{  
                """
    cp = nltk.RegexpParser(grammar) 
    result = []
    tree = cp.parse(pos)
    for subtree in tree.subtrees():
        if subtree.label()=='HEAD':
            phrase = u' '.join([word for word,pos in subtree.leaves()])
            phrase_list = phrase.split()
            if len(phrase_list)>1:
                ph = phrase_list[-1]
            else:
                ph = phrase
            result.append(ph)
            #print subtree.label()+':'+phrase+'->'+ph       
    return result

In [5]:
headwords = []
year_pattern = re.compile(r'(^[0-9]{4}s?$)')
person_pattern = re.compile(r'(^(the )?([A-Z]+[a-zA-Z\']+ ?((&|and|of) )?)+$)')
number_pattern = re.compile(r'([0-9]+)')
the_pattern = re.compile(r'(^the)')

year_headword = {}
organization_headword = {}
person_headword = {}
number_headword = {}
location_headword = {}

count = 0
for article in train:
    print count
    count += 1
    qas = article['qa']
    sentences = article['sentences']
    token_sentences = copy.deepcopy(sentences)
    for i in range(len(sentences)):
        token_sentences[i] = nltk.word_tokenize(token_sentences[i])
    ner_sentences = st.tag_sents(token_sentences)
    parse_ner_sentences = parse_NER(ner_sentences)
    #print parse_ner_sentences
    for qa in qas:
        rank = {}
        ner_sentence = parse_ner_sentences[qa['answer_sentence']]
        for word in nltk.word_tokenize(qa['answer']):
            for w,t in ner_sentence:
                if t != u'O' and w == word:
                    rank[t] = rank.get(t,0) + 1
        result = sorted(rank.items(), lambda x, y: cmp(x[1], y[1]), reverse=True)
        if result != []:        
            kind = result[0][0]
            text = qa['question']
            if kind == u'NUMBER':
                for word in getHeadWord(text):
                    number_headword[word] = number_headword.get(word,0) + 1
            elif kind == u'YEAR':
                for word in getHeadWord(text):
                    year_headword[word] = year_headword.get(word,0) + 1
            elif kind == u'ORGANIZATION':
                for word in getHeadWord(text):
                    organization_headword[word] = organization_headword.get(word,0) + 1
            elif kind == u'PERSON':
                for word in getHeadWord(text):
                    person_headword[word] = person_headword.get(word,0) + 1
            elif kind == u'LOCATION':
                for word in getHeadWord(text):
                    location_headword[word] = location_headword.get(word,0) + 1
                

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
year_headword_edit = copy.deepcopy(year_headword)
organization_headword_edit = copy.deepcopy(organization_headword)
person_headword_edit = copy.deepcopy(person_headword)
number_headword_edit = copy.deepcopy(number_headword)
location_headword_edit = copy.deepcopy(location_headword)

NameError: name 'year_headword' is not defined

In [60]:
year_headword = copy.deepcopy(year_headword_edit)
organization_headword = copy.deepcopy(organization_headword_edit)
person_headword = copy.deepcopy(person_headword_edit)
number_headword = copy.deepcopy(number_headword_edit)
location_headword = copy.deepcopy(location_headword_edit)

In [61]:
threshold = 1

for (w,n) in year_headword.items():
    if w in organization_headword:
        if n >= organization_headword.get(w):
            del organization_headword[w]
        else:
            del year_headword[w]
for (w,n) in year_headword.items():
    if w in person_headword:
        if n >= person_headword.get(w):
            del person_headword[w]
        else:
            del year_headword[w]
for (w,n) in year_headword.items():
    if w in number_headword:
        if n >= number_headword.get(w):
            del number_headword[w]
        else:
            del year_headword[w]
for (w,n) in year_headword.items():
    if w in location_headword:
        if n >= location_headword.get(w):
            del location_headword[w]
        else:
            del year_headword[w]
            
for (w,n) in organization_headword.items():
    if w in person_headword:
        if n >= person_headword.get(w):
            del person_headword[w]
        else:
            del organization_headword[w]
for (w,n) in organization_headword.items():
    if w in number_headword:
        if n >= number_headword.get(w):
            del number_headword[w]
        else:
            del organization_headword[w]
for (w,n) in organization_headword.items():
    if w in location_headword:
        if n >= location_headword.get(w):
            del location_headword[w]
        else:
            del organization_headword[w]
            
for (w,n) in person_headword.items():
    if w in number_headword:
        if n >= number_headword.get(w):
            del number_headword[w]
        else:
            del person_headword[w]
for (w,n) in person_headword.items():
    if w in location_headword:
        if n >= location_headword.get(w):
            del location_headword[w]
        else:
            del person_headword[w]

for (w,n) in number_headword.items():
    if w in location_headword:
        if n >= location_headword.get(w):
            del location_headword[w]
        else:
            del number_headword[w]
            
for (w,n) in year_headword.items():
    if n < threshold:
        del year_headword[w]
for (w,n) in organization_headword.items():
    if n < threshold:
        del organization_headword[w]
for (w,n) in person_headword.items():
    if n < threshold:
        del person_headword[w]
for (w,n) in number_headword.items():
    if n < threshold:
        del number_headword[w]
for (w,n) in location_headword.items():
    if n < threshold:
        del location_headword[w]


location_list = location_headword.keys()
number_list = number_headword.keys()
organization_list = organization_headword.keys()
name_list = person_headword.keys()
year_list = year_headword.keys()

print 'Year: ',year_headword.keys()
print 'Organ: ',organization_headword.keys()
print 'Person: ',person_headword.keys()
print 'Number: ',number_headword.keys()
print 'Location: ',location_headword.keys()

Year:  [u'protests', u'frame', u'constellations', u'years', u'born', u'election', u'year', u'yer', u'decade', u'span', u'Bowers', u'rains', u'frog', u'crime', u'window', u'Bower', u'America', u'decades', u'preseason', u'attention', u'Dewar', u'Microsoft', u'require', u'Period', u'influx', u'periods']
Organ:  [u'writings', u'AGG', u'Jayapala', u'dynasty', u'protest', u'Lodge', u'NYC', u'relationships', u'Foundation', u'patches', u'invaders', u'sub-group', u'teaching', u'League', u'risk', u'Industry', u'rise', u'slaveholder', u'pigment', u'GE', u'Digimon', u'Henry', u'school', u'prize', u'SNL', u'companies', u'someplaces', u'samurai', u'vector', u'Nepalese', u'IXs', u'Occitan', u'Jing', u'Bavaria', u'force', u'leaders', u'Strasbourg', u'Festival', u'Mizrach', u'nail', u'follow', u'monster', u'employ', u'organisms', u'offshoots', u'commemorating', u'orchestra', u'sung', u'SAGW', u'fossil', u'churches', u'Amphetamine', u'admiral', u'VLW', u'evolutionism', u'Neptune', u'Branch', u'men', u'a

In [6]:
def BM25_n(articles_index,k1,k2,b,n):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    correct_id = []
    correct_id_weight = []
    cor = 0.0
    for index in range(len(total_question_bow[articles_index])):
        poss_results = find_max_n_sentences(articles_index,index,k1,k2,b,n)
        guess_ids = []
        weights = []
        for sent in poss_results:
            guess_ids.append(sent[0])
            weights.append(sent[1])
        answer_id = answer_id_list[articles_index][index]
        if answer_id in guess_ids:
            cor += 1
        correct_id.append(guess_ids)
        correct_id_weight.append(weights)
    accuracy = cor/total_queries
    return correct_id,correct_id_weight,accuracy

def find_max_n_sentences(articles_index,index,k1,k2,b,n):
    query_dict = total_question_bow[articles_index][index]
    scores = {}
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        scores[index] = score
    scores = sorted(scores.items(), key=operator.itemgetter(1),reverse=True)[:n]
    return scores

In [15]:
k1_list = [0.78]
k2_list = [0]
b_list = [0.5]

result_sentences = []
result_sentences_weight = []
test_length = len(dev)
for amt in [4]:
    accuracy = 0.0
    for k1 in k1_list:
        for k2 in k2_list:
            for b in b_list:
                for i in range(0,test_length):
                    #the amount of extract sentences
                    amount = amt
                    correct_id,correct_id_weight,i_accuracy = BM25_n(i,k1,k2,b,amount)
                    accuracy += i_accuracy
                    result_sentences.append(correct_id)
                    result_sentences_weight.append(correct_id_weight)
                average_accuracy = accuracy/test_length
                print average_accuracy

0.831647500194


In [11]:
# Accuracy of BM25 with k1: 1.1  k2: 0  b: 0.18
# 0.683639289744

In [16]:
year_list = [u'protests', u'frame', u'constellations', u'years', u'born', u'election', u'year', u'yer', u'decade', u'span', u'Bowers', u'rains', u'frog', u'crime', u'window', u'Bower', u'America', u'decades', u'preseason', u'attention', u'Dewar', u'Microsoft', u'require', u'Period', u'influx', u'periods']
# name_list = [u'composer', u'figure', u'president', u'architect', u'singer', u'writer', u'artist', u'author', u'ruler', u'philosopher', u'leader', u'Darwin', u'actor']
name_list = [u'Neuman', u'founder', u'manager', u'Morovic', u'talks', u'chair', u'compose', u'Assemblyman', u'passage', u'painters', u'logician', u'painter', u'idealist', u'include', u'woman', u'Cid', u'KSOG', u'Topnail', u'affect', u'prefect', u'scholar', u'knight', u'Hameidi', u'villa', u'prediction', u'thinker', u'Shakespeare', u'insertion', u'cartel', u'panda', u'atman', u'captain', u'Deleuze', u'Bilal', u'crush', u'Popper', u'scientists', u'reporter', u'witch', u'legend', u'talent', u'biographies', u'shift', u'deers', u'ALS', u'reds', u'straw', u'Laemmle', u'composer', u'franca', u'algae', u'discourses', u'conquistador', u'Charity', u'musician', u'Menzies', u'dogma', u'orator', u'sculptor', u'Neumann', u'Pope', u'Yousong', u'German', u'equation', u'Politicians', u'concepts', u'meet', u'figure', u'attribute', u'instrumentalism', u'tax', u'Dominicans', u'botanist', u'monuments', u'writers', u'answer', u'Zealand', u'algebra', u'variant', u'president', u'man', u'neopragmatist', u'doctor', u'Luke', u'neglect', u'producer', u'Anatolia', u'setback', u'Xianzhong', u'Rotten', u'Jah', u'deity', u'mayor', u'Stein', u'acknowledge', u'relativity', u'degrees', u'ally', u'Dominican', u'descendants', u'mention', u'President', u'front', u'Friar', u'Dominic', u'investor', u'sects', u'investiture', u'mathematician', u'identifier', u'Martel', u'decree', u'novels', u'health', u'strategist', u'brewery', u'Theater', u'ofen', u'grapes', u'explorers', u'ex-speaker', u'congressman', u'musicians', u'castle', u'copper', u'Rastrakuta', u'student', u'researcher', u'waves', u'Historian', u'Asia', u'navigator', u'engineer', u'top', u'historian', u'teacher', u'saint', u'convention', u'friend', u'subplot', u'reinforcements', u'Mozart', u'expatriate', u'believe', u'Nobunaga', u'Lenin', u'performers', u'recording', u'Kaunitz', u'architect', u'feeling', u'symmetries', u'makhtesh', u'Meinhard', u'philosphers', u'ballerina', u'warriors', u'Minister', u'doors', u'play', u'transactions', u'rappers', u'frontman', u'pharaoh', u'architects', u'rehearsal', u'Priest', u'rationale', u'icon', u'Bridge', u'singer', u'seize', u'conquests', u'professor', u'velocity', u'physics', u'scream', u'detective', u'pope', u'authors', u'contemporary', u'traitor', u'industrialist', u'economist', u'Powell', u'principal', u'archbisoph', u'justice', u'writer', u'French', u'employee', u'meant', u'Rommel', u'executive', u'king', u'artist', u'rapper', u'extremists', u'husband', u'grain', u'Toppman', u'locale', u'Archbishop', u'AEA', u'see', u'churchman', u'Babylon', u'expert', u'Noriega', u'Tillcar', u'Gautama', u'Rajasthanis', u'importance', u'physiologist', u'Bronn', u'monk', u'officer', u'Tipitaka', u'mausoleum', u'J.C.B', u'knee-deep', u'Elizabethan', u'discoverer', u'Kermes', u'biographer', u'zinc', u'Hamilton', u'Scout', u'poet', u'Singer', u'Adams', u'Georgie', u'present', u'politician', u'Deputy', u'governor', u'monumnet', u'aid', u'Spiro', u'resident', u"O'Daniel", u'commander', u'player', u'Bush', u'arenas', u'Admiral', u'synagogue', u'author', u'sociologist', u'member', u'negotiations', u'monarch', u'inventors', u'functions', u'director', u'senator', u'Polya', u'No.46', u'bearer', u'settlements', u'actress', u'prince', u'Bahrain', u'candidate', u'photographer', u'character', u'advancement', u'collections', u'usage', u'humanity', u'match', u'chairman', u'complaint', u'fate', u'Restaurant', u'ruler', u'royalty', u'judge', u'clan', u'Jehovah', u'Creed', u'broadcaster', u'journalist', u'prosector', u'sketch', u'composers', u'Lee', u'philosopher', u'Oberhauser', u'Sulla', u'commentator', u'leader', u'biopic', u'Stengers', u'refers', u'beaches', u'Gelug', u'sister', u'Reactor', u'sighting', u'neighbor', u'UD', u'tomb', u'emperor', u'Kendals', u'Boulevard', u'Schnell', u'secondmost', u'son', u'Iraqi', u'roads', u'herbalist', u'fought', u'Simpson', u'pitcher', u'specialty', u'Archivist', u'Luciano', u'Minaj', u'Helena', u'abolitionist', u'general', u'Cabot', u'Darwin', u'dealer', u'Imam', u'englishman', u'Jr.', u'XXIII', u'actor', u'politicians', u'satirist', u'Lodensteijn', u'star', u'Karim', u'chairperson', u'mosaics', u'minister']
number_list = [u'bagpipe', u'similarity', u'lack', u'resistance', u'month', u'highways', u'deadline', u'ammonia', u'bars', u'increase', u'zone', u'icons', u'sniper', u'Buffett', u'shaft', u'elevations', u'Origin', u'level', u'solution', u'Index', u'escalation', u'Europes', u'estimates', u'rate', u'cost', u'scroe', u'machines', u'sun', u'Merkel', u'anniversary', u'thickness', u'Socio', u'public', u'temperatures', u'ration', u'Libya', u'Brasilia', u'error', u'hours', u'operating', u'tuition', u'precursor', u'amount', u'advertising', u'celebrants', u'punishment', u'criticism', u'diameter', u'namee', u'pavement', u'regiment', u'Hanover', u'federlism', u'Nigeria', u'total', u'tools', u'standing', u'attendance', u'beginning', u'guideline', u'asses', u'spike', u'rainfall', u'integers', u'abundance', u'involve', u'intake', u'hour', u'Locke', u'half-life', u"'Pra", u'ECCE', u'revenues', u'ranking', u'share', u'numbers', u'slip', u'Years', u'AFL', u'axis', u'Welsh', u'methodology', u'earnings', u'significance', u'fare', u'Expedition-1', u'clone', u'spot', u'aborigines', u'representations', u'Churchill', u'Lilius', u'Tennessee', u'temp', u'Greece', u'turnover', u'wavelengths', u'calibre', u'timing', u'willow', u'Delhi', u'coordinates', u'crew', u'Spain', u'reveal', u'commitment', u'EYS', u'Gabon', u'effects', u'slippage', u'day', u'dielectric', u'continuum', u'liberia', u'Globe', u'composition', u'magnitude', u'wage', u'accounts', u'subset', u'prohibit', u'weight', u'gun', u'pecentage', u'yield', u'navy', u'wavelength', u'margins', u'collector', u'FlaK', u'dominates', u'space', u'Frued', u'medicine', u'phenolic', u'mechanisms', u'isotope', u'Tucson', u'formation', u'estimate', u'turnout', u'widths', u'positioning', u'Arsenal', u'length', u'Montevideo', u'margin', u'retail', u'speeds', u'hydrides', u'number', u'rank', u'date', u'Shiraz', u'downside', u'specificity', u'Genesis', u'size', u'RMP', u'Diamond', u'Iran', u'temperature', u'plains', u'Rakhine', u'codons', u'rates', u'identity', u'calculations', u'bounty', u'percentage', u'ceiling', u'statement', u'season', u'Guinea-Bissau', u'rifle', u'population', u'distance', u'hike', u'KU', u'argument', u'Odeon', u'need', u'concentration', u'Valley', u'Eurozone', u'angle', u'instance', u'post-punk', u'materials', u'accuracy', u'Jones', u'price', u'measures', u'Frederick', u'Minds', u'mouth', u'pair', u'seasons', u'drought', u'damage', u'D+', u'average', u'GDP', u'request', u'newcomers', u'emperature', u'motorways', u'bracket', u'Springs', u'quantity', u'rating', u'restriction', u'brother', u'threshold', u'fine', u'find', u'ratio', u'Austin', u'winner', u'proportion', u'bases', u'Historians', u'Liberia', u'incentives', u'vowels', u'GPA', u'reporters', u'OMOV', u'stability', u'EQ', u'circulation', u'chirigotas', u'grades', u'sidearm', u'allegation', u'contain', u'release', u'resonance', u'requirement', u'set', u'edition', u'Vedanta', u'radius', u'lifespan', u'hole', u'score', u'Huxley', u'UNAIDS', u'Glimm', u'Browning', u'blame', u'efficiency', u'haplogroup', u'debt', u'isotopes', u'iPod', u'Thomson', u'auctions', u'Sepphoris', u'expression', u'load', u'suggest', u'point', u'snowfall', u'height', u'Was', u'boat', u'decline', u'Plan', u'make-up', u'trinity', u'life', u'efficacy', u'measles', u'gas', u'gap', u'expenditure', u'orchestras', u'divisions', u'landing', u'estimations', u'budget', u'value', u'Guptas', u'Cyprus', u'amoun', u'century', u'Imperial', u'neurons', u'brake', u'centennial', u'Valencia', u'Phillipines', u'Product', u'pay', u'quirk', u'parts', u'biologists', u'satellite', u'wires', u'centuries', u'valuation', u'rain', u'levels', u'Burmanisation', u'cycle', u'percentages', u'object', u'honor', u'savings', u'spectator', u'Jama', u'dose', u'speed', u'announcement', u'density', u'absorption', u'MYS', u'munition', u'demographic', u'Dome', u'improvement', u'BCE', u'balloons', u'monotheists', u'elevation', u'myth', u'capacities', u'measurement', u'Roberts', u'traffic', u'variants', u'precipitation', u'Flair', u'wavelenght', u'London', u'output', u'percentsge', u'liquidation', u'Brandenburg', u'shortcoming', u'Christensen', u'dimensions', u'measurements', u'audience', u'per', u'cloud', u'inclination', u'School', u'incidence', u'run', u'warlords', u'Beringia', u'perpendicular', u'Times', u'post', u'Loop', u'Hyderabad', u'extension', u'months', u'ranges', u'Plymouth', u'range', u'expectancy', u'coverage', u'decrease', u'capable', u'rounds', u'diameters', u'altitude', u'Entertainment', u'custom', u'inconsistency', u'fraction', u'crews', u'Counties', u'tally', u'Detroit', u'count', u'Invader', u'characteristic', u'Knesset', u'Africa', u'maximum', u'LER', u'limit', u'Boas', u'constant', u'sale', u'logo', u'lifetime', u'commentators', u'MSA', u'%', u'floor', u'stake', u'percent', u'portion', u'income', u'users', u'Suarez', u'Arnold', u'chance', u'scale', u'Rooney', u'Earth', u'Myanmar', u'age', u'Comcast', u'depth', u'mass', u'diagnosis', u'time', u'Bombycilla', u'resolution', u'profits', u'demodulator']
location_list = [u'Canada', u'Manchuse', u'tombs', u'archaeologists', u'gang', u'Capital', u'settlement', u'Somaliland', u'birthplace', u'archdiocese', u'EST5EDT', u'catastrophe', u'providence', u'mosque', u'lawmakers', u'road', u'federation', u'connects', u'miners', u'Federalism', u'sunk', u'harbor', u'Arabs', u'venue', u'Caesar', u'settlers', u'garden', u'neighborhood', u'streets', u'rivers', u'aquifer', u'State', u'street', u'Korea', u'constitution', u'superpower', u'capital', u'Gampo', u'lacquer', u'Festino', u'compilation', u'Utrecht', u'path', u'earthquake', u"'Scullers", u'Rodman', u'snowbirds', u'derby', u'Continent', u'Wonder', u'Tuscon', u'Yangon', u'gateway', u'Australia', u'Hub', u'counties', u'possession', u'Bei', u'populations', u'towers', u'statutes', u'Confederate', u'accessories', u'trail', u'Gorbachev', u'town', u'Affair', u'tunnel', u'Sweden', u'Ashkenaz', u'Kuznetsov', u'mall', u'freedom', u'proof', u'estate', u'CAR', u'states', u'Forbes', u'dieseases', u'acts', u'Congo', u'bricks', u'palaces', u'beach', u'capitol', u'economy', u'mas', u'spelling', u'Petersen', u'geographies', u'manpower', u'Japanese', u'Middle', u'Party', u'Ptolemy', u'Purple', u'Kushans', u'bombing', u'disagreed', u'attraction', u'complex', u'combination', u'Bay', u'belt', u'Hospital', u'Tower', u'lineage', u'Sam', u'tiers', u'ports', u'legions', u'Communists', u'square', u'Organisation', u'tenth', u'thoroughfares', u'Michigan', u'factory', u'seaport', u'hill', u'expressway', u'G', u'Smiles', u'base', u'Architecture', u'conflicts', u'district', u'submarines', u'Nepal', u'Goring', u'Horn', u'connotation', u'Lama', u'relate', u'municipality', u'primary', u'comprises', u'Gilbert', u'city', u'metalwork', u'seat', u'relations', u'PVA', u'Entente', u'dogs', u'ecotourist', u'hub', u'hotel', u'Podgorica', u'part', u'Rajasthani', u'inland', u'monarchies', u'nations', u'steet', u'Zaire', u'Peter', u'powers', u'supercontinent', u'bridge', u'motivation', u'palace', u'suspicious', u'mine', u'headland', u'Yama', u'mountain', u'jurisdictions', u'country/territory', u'zoo', u'Congregation', u'Germany', u'holders', u'Bronck', u'province', u'combat', u'monastery', u'synonym', u'neighborhoods', u'alarm', u'drive', u'freeway', u'Canon', u'Pakistan', u'Byzantines', u'Union', u'bring', u'Quasm', u'lilangeni', u'soviets', u'Kahuta', u'ground', u'Province', u'subgenre', u'terminal', u'notes', u'get', u'stop', u'coast', u'River', u'areas', u'hall', u'scene', u'countries', u'Washington', u'bison', u'river', u'Mongols', u'sea', u'John', u'constellation', u'langauge', u'Park', u'state', u'Freedom', u'infrastructure', u'confusion', u'Luftwaffe', u'southeast', u'hypothesis', u'Shaokun', u'outer', u'cities', u'undermine', u'Kangra', u'goddess', u'installation', u'restaurant', u'fault', u'region', u'declare', u'marchers', u'KMC', u'royal', u'colony', u'community', u'canyon', u'village', u'ADGB', u'provinces', u'airlines', u'stadium', u'Reporters', u'borders', u'territory', u'trend', u'location', u'direction', u'Moldova', u'World', u'independence', u'plain', u'heartland', u'Jurisdiction', u'Reagan', u'US-30', u'country', u'Taiwan', u'Avenue', u'decolonization', u'site', u'surface', u'Wood', u'partner', u'waterway', u'alliances', u'capture', u'borough', u'italy', u'border', u'airport', u'Coastline', u'Island', u'commune', u'Shaivism', u'Persis', u'hand', u'suburb', u'Airport', u'nation', u'database', u'Miller', u'ocean', u'States', u'centres', u'camps', u'comprise', u'gods', u'Mexicans', u'Territory', u'continent', u'atoll', u'blow', u'conclude', u'lake', u'Victoria', u'airfield', u'Connecticut', u'plateau', u'homeland', u'Front', u'confederacy', u'kingdom', u'landings', u'Enlai', u'protesters', u'Drake', u'Ortega', u'harbour', u'railroad', u'pier', u'Reduit', u'county', u'Albania', u'Titograd', u'exclave', u'tribes', u'NAFTA', u'reaches', u'Perry', u'spring', u'Qiang', u'protectorate', u'recognition', u'faction', u'councel', u'Mahan', u'intersections', u'temple', u'Kota', u'metropolis', u'Pangaea', u'Shibell', u'lakes', u'Range', u'convent', u'om', u'island', u'city-states', u'militia', u'islands', u'basin', u'Samanid', u'Kazusanosuke', u'promiss', u'lands', u'transistron', u'Newars', u'revolutionaries', u'Abdullah', u'laborers', u'right', u'promise', u'canal', u'area', u'Ming', u'Serval', u'valley', u'Museum', u'mountains', u'north', u'continents', u'January', u'hemisphere', u'Namibia', u'hottest', u'blockading', u'sutras', u'partitioning', u'terminus', u'places', u'Wall', u'Britain', u'Prussians', u'Terminal', u'Edward', u'ecoregion', u'priory', u'burrough', u'Heinrich', u'Tsongas', u'exporter', u'Daytona', u'vocabulary', u'annex', u'Country', u'April', u'peak', u'Retable', u'Melborne', u'land', u'Desert', u'Mission', u'mercenary', u'Street', u'gardens']
# number_list = [u'month', u'increase', u'level', u'rate', u'cost', u'amount', u'Nigeria', u'share', u'date', u'Greece', u'day', u'Tucson', u'temperature', u'length', u'number', u'rank', u'size', u'percentage', u'season', u'population', u'price', u'average', u'GDP', u'ratio', u'score', u'point', u'value', u'century', u'centuries', u'speed', u'density', u'range', u'fraction', u'Detroit', u'limit', u'percent', u'portion', u'income', u'age', u'time']
# location_list = [u'venue', u'neighborhood', u'street', u'capital', u'airport', u'town', u'states', u'district', u'city', u'part', u'province', u'areas', u'countries', u'river', u'state', u'cities', u'region', u'territory', u'location', u'country', u'site', u'nation', u'continent', u'kingdom', u'island', u'road', u'area', u'county']

In [17]:
location_list = location_list + ['country','county','district','city']
number_list = number_list + ['version','size','msa','far','much','ration','time','many','population','large','percent','average','day','decade','big','long']
name_list = name_list + ['name','center','president','denominations','denomination','film','broadcaster','pitcher','commentator']
year_list = year_list + ['years','year','era']

In [25]:
location_list = ['country','county','district','city']
number_list = ['version','size','msa','far','much','ration','time','many','population','large','percent','average','day','decade','big','long']
name_list = ['name','center','president','denominations','denomination','film','broadcaster','pitcher','commentator']
year_list = ['years','year','era']

In [18]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re
import sys

def transfer_pos_question(pos):
    new_pos = []
    for (word,wtype) in pos:
        if word.lower() == 'what' or word.lower() == 'what\'s':
            new_pos.append((word,'WHAT'))
        elif word.lower() == 'do' or word.lower() == 'does' or word.lower() == 'did':
            new_pos.append((word,'DO'))
        elif word in number_list:
            new_pos.append((word,'NUMBER'))
        elif word in year_list:
            new_pos.append((word,'YEAR'))
        elif word in name_list:
            new_pos.append((word,'NAME'))
        elif word in location_list:
            new_pos.append((word,'LOC'))
        elif word.lower() == 'is' or word.lower() == 'was' or word.lower() == 'are' or word.lower() == 'were' or word.lower() == 'be':
            new_pos.append((word,'BE'))
        elif word.lower() == 'when':
            new_pos.append((word,'WHEN'))
        elif word.lower() == 'where':
            new_pos.append((word,'WHERE'))
        elif word.lower() == 'can':
            new_pos.append((word,'CAN'))
        elif word.lower() == 'how':
            new_pos.append((word,'HOW'))
        elif word.lower() == 'who' or word.lower() == 'whom' or word.lower() == 'whose'  or word.lower() == 'whos':
            new_pos.append((word,'WHO'))
        elif word.lower() == 'which':
            new_pos.append((word,'WHICH'))
        elif word.lower() == 'define':
            new_pos.append((word,'DEFINE'))
        elif word.lower() == 'should':
            new_pos.append((word,'SHOULD'))
        elif word.lower() == 'why' or word.lower() == 'wy':
            new_pos.append((word,'WHY'))
        else:
            new_pos.append((word,wtype))
    return new_pos

def get_continuous_chunks(text):
    t = copy.deepcopy(text)
    pos = pos_tag(nltk.word_tokenize(t))
    pos = transfer_pos_question(pos)
    #print pos
    grammar = r"""
                WHAT: 
                    {<WHAT>}
                    {<WHICH>}
                    {<DEFINE>}
                WHO:
                    {<WHO>}
                    {<WHAT><BE>?<DT>?<JJ|RB>*<NAME>}
                    {<WHAT><JJ|RB>*<NN>+<NAME>}
                NUMBER:
                    {<WHICH><NUMBER>}
                    {<HOW><NUMBER>}
                    {<WHAT><BE>?<DT>?<JJ>?<NN>*<JJ>?<NUMBER>}
                WHEN:
                    {<WHICH><YEAR>}
                    {<WHAT><BE>?<DT>?<JJ>?<NN>*<JJ>?<YEAR>}
                    {<WHEN>}
                WHERE:
                    {<WHERE>}
                    {<WHAT><LOC>}
                    {<WHAT><BE>?<DT>?<RBS>?<JJ>?<LOC>}
                HOW:
                    {<CAN>}
                    {<DO>}
                    {<SHOULD>}
                    {<WHY>}
                    {<HOW>}

                """
    cp = nltk.RegexpParser(grammar) 
    result = []
    tree = cp.parse(pos)
    #record the position of pos
    #print pos
    flag = 0
    for subtree in tree.subtrees():
        if subtree.label() != 'S':
            phrase = u''
            for word,pos in subtree.leaves():
                if word == ',':
                    phrase = phrase + word
                else:
                    phrase = phrase + u' '
                    phrase = phrase + word
            result.append((subtree.label(),phrase[1:]))         
    return result

In [19]:
def transfer_pos_sentence(pos):
    new_pos = []
    
    for (word,wtype) in pos:
        time_pattern = re.compile(r'[0-9]{4}s?')
        if word.lower() == 'and' or word.lower() == 'or':
            new_pos.append((word,'POSICC'))
        elif word.lower() == 'with':
            new_pos.append((word,'WITH'))
        elif word.lower() == 'a' or word.lower() == 'an':
            new_pos.append((word,'A'))
        elif word == '"':
            new_pos.append((word,'"'))
        elif word == 'minutes' or word == 'March':
            new_pos.append((word,'TIME'))
        elif time_pattern.search(word):
            new_pos.append((word,'TIME'))
        elif word == 'around':
            new_pos.append((word,'AROUND'))
        elif word == 'mm':
            new_pos.append((word,'CD')) 
        else:
            new_pos.append((word,wtype))
    return new_pos

In [20]:
def screen_out_answer_WHAT(result):
    answer_list = []
    for wtype, word in result:
        answer_list.append(word)
    return answer_list

In [21]:
def rank_rule_1(entity,query):
    #lower scores for content words also appear in the query
    count = 0
    length = len(entity)
    for word in entity:
        word = lemmatize(word.lower())
        if word not in stopwords:
            if word in query:
                count += 1
    if length == 0:
        score = 0
    else:
        score = 1 - float(count)/length
    return score

In [22]:

def get_open_class_words(query):
    result = []
    for index in range(len(query)):
        if query[index] not in stopwords:
            if query[index] not in string.punctuation:
                result.append(query[index])
    return result

In [23]:
def rank_rule_3(answer_sentence,sentence,entity,query):
    #higher scores for closer distance between an entity and the headword
    #step 1: using a filter to extract "useful" open-class words
    results = get_open_class_words(query)
    sent = sentence
    original_sent = answer_sentence
    entity_loc = []
    query_loc = []
    for word in entity:
        if word in original_sent:
            entity_loc.append(original_sent.index(word))
    for q in results:
        if q in sent:
            query_loc.append(sent.index(q))
    min_dist = len(original_sent)
    if query_loc != []:
        for i in query_loc:
            for j in entity_loc:
                dist = abs(i - j)
                if dist < min_dist:
                    min_dist = dist
                    
    return 1 - float(min_dist)/len(original_sent)

In [24]:
def get_continuous_chunks_sentence(text,texttype):
    t = copy.deepcopy(text)
    pos =  pos_tag(nltk.word_tokenize(t))
    if texttype==0:
        #WHAT
        pos = transfer_pos_sentence(pos)
        grammar = r"""
                    J:
                        {<JJ.*><VBN>}
                        {<JJ.*><POSICC><JJ.*>}   
                        {<JJ.*>+}
                        {<NN.*><POS>}
                    N:
                        {<CD>+<NN.*>}
                        {<A>?<NN.*>?<J>?<NN.*>+}
                        <\">{<A>?<J>?<NN.*>+}<\">
                    COMBON:
                        {(<N><,>)*<N><,>?<POSICC><N>}
                    NWC:
                        {<N><WITH><COMBON>}
                    """
    cp = nltk.RegexpParser(grammar) 
    result = []
    poss = copy.deepcopy(pos)
    tree = cp.parse(pos)
    #record the position of pos
    flag = 0
    for i in range (len(tree)):   
        if type(tree[i]) != tuple:
            subtree = tree[i]
            if texttype==0 and subtree.label() != 'S':
                phrase = u''
                for word,pos in subtree.leaves():
                    if word == ',':
                        phrase = phrase + word
                    else:
                        phrase = phrase + u' '
                        phrase = phrase + word
                result.append((subtree.label(),phrase[1:]))
                #print subtree.label(),phrase
            elif subtree.label() != 'S':
                phrase = u' '.join([word for word,pos in subtree.leaves()])
                result.append((subtree.label(),phrase))
    return poss,result

In [27]:
count_contains = 0
count_similar = 0.0
count_correct = 0
total_question = 0

for index in range(5):#range(len(train)):
    print "processing: ",index
    article = dev[index]
    qas = article['qa']
    sentences = article['sentences']
    token_sentences = copy.deepcopy(sentences)
    for i in range(len(sentences)):
        token_sentences[i] = nltk.word_tokenize(token_sentences[i])
    ner_sentences = st.tag_sents(token_sentences)
    parse_ner_sentences = analyse_NER(parse_NER(ner_sentences))
    total_question += len(qas)
    for i in range(len(qas)):
        flag = False
        answer_list = []
        qa = qas[i]
        real_answer = qa['answer']
        real_answer_sentence_id = qa['answer_sentence']
        answers = []
        scs = []
        amount = 1
        #the amount of extract sentences
        for ii in range(amount):
            if result_sentences_weight[index][i][ii] != 0:
                weight = float(result_sentences_weight[index][i][ii])/sum(result_sentences_weight[index][i])
            else:
                weight = 0
            answer_sentence = sentences[result_sentences[index][i][ii]]
            answer_sentence_id = result_sentences[index][i][ii]
            text_question = qa['question']
            result = get_continuous_chunks(text_question)

            #print pos
            if result != []:
                
                wtype,word = result[0]
                #print wtype,qa['question']
                #print 'Answer: ', qa['answer']
                if wtype == 'NUMBER':
                    #print 'NUMBER',qa['question']
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],1)
                    if qa['answer'] in answer_list:
                        flag = True
                        total_count += 1
                elif wtype == 'WHO':
                    #print 'WHO',qa['question']
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],0)
                    if answer_list == []:
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],2)
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                    if qa['answer'] in answer_list:
                        flag = True
                        total_count += 1
                elif wtype == 'WHERE':
                    #print 'WHERE',qa['question']
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],2)
                    if answer_list == []:
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],0)
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                    if qa['answer'] in answer_list:
                        flag = True
                        total_count += 1
                elif wtype == 'WHEN':
                    #print 'WHEN',qa['question']
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],4)
                    if qa['answer'] in answer_list:
                        flag = True
                        total_count += 1
                else:
                    #WHAT
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],3)
#                     if answer_list == []:
#                         answer_sentence = sentences[answer_sentence_id]
#                         wpos,wresult = get_continuous_chunks_sentence(answer_sentence,0)
#                         answer_list = screen_out_answer_WHAT(wresult)
#                         answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],2)
#                         answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],0)
#                         answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],1)
#                         answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],4)
                        
                    if qa['answer'] in answer_list:
                        flag = True
                        total_count += 1
                #print answer_list
                #print sentences[answer_sentence_id]
            else:
                #print 'None', qa['question']
                #print 'Answer: ', qa['answer']
                answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],3)
#                 if answer_list == []:
#                     answer_sentence = sentences[answer_sentence_id]
#                     wpos,wresult = get_continuous_chunks_sentence(answer_sentence,0)
#                     answer_list = screen_out_answer_WHAT(wresult)
                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1         
            if answer_list != []:
                count_contains += 1
                query = copy.deepcopy(text_question)
                query = nltk.word_tokenize(query)
                #token the answer sentence and copy it for further usage
                answer_sentence = sentences[answer_sentence_id]
                answer_sentence = nltk.word_tokenize(answer_sentence)
                sentence = copy.deepcopy(answer_sentence)

                for query_index in range(len(query)):
                    query[query_index] = lemmatize(query[query_index].lower())

                for sent_index in range(len(sentence)):
                    sentence[sent_index] = lemmatize(sentence[sent_index].lower())

                scores_1 = []
                scores_3 = []
                scores = []

                for entity in answer_list:
                    entity = nltk.word_tokenize(entity) 
                    score1 = rank_rule_1(entity,query)
                    scores_1.append(score1)
                    #answer_sentence is the original version and sentence is preprocessed
                    score3 = rank_rule_3(answer_sentence,sentence,entity,query)
                    scores_3.append(score3)
                    w1 = 0.7
                    w3 = 1 - w1
#                     if score1 == 0:
#                         score3 = 0
                    total = w1 * score1 + w3 * score3
                    scores.append(total)
                answer = answer_list[scores.index(max(scores))]
                sc = max(scores)*weight
            
            else:
                answer = answer_sentence
                sc = 0
            answers.append(answer)
            scs.append(sc)
        ans = answers[scs.index(max(scs))]
        
        if answer == qa['answer']:
            count_correct += 1
            count_similar += 1
        else:
            answer_token = nltk.word_tokenize(answer)
            c_answer_token = nltk.word_tokenize(qa['answer'])
            if len(answer_token) > len(c_answer_token):
                mark_answer = 0
                mark_c_answer = 0
                match = 0
                while(True):
                    if answer_token[mark_answer] == c_answer_token[mark_c_answer]:
                        mark_answer += 1
                        mark_c_answer += 1
                        match += 1
                    else:
                        mark_answer -= (match-1)
                        match = 0
                        mark_c_answer = 0
                    if match == len(c_answer_token):
                        #print answer,'&',qa['answer']
                        count_similar += float(len(c_answer_token))/len(answer_token)
                        break
                    if mark_answer >= len(answer_token):
                        break                                        
            else:
                mark_answer = 0
                mark_c_answer = 0
                match = 0
                while(True):
                    if answer_token[mark_answer] == c_answer_token[mark_c_answer]:
                        mark_answer += 1
                        mark_c_answer += 1
                        match += 1
                    else:
                        mark_c_answer -= (match-1)
                        match = 0
                        mark_answer = 0
                    if match == len(answer_token):
                        #print answer,'&',qa['answer']
                        count_similar += float(len(answer_token))/len(c_answer_token)
                        break
                    if mark_c_answer >= len(c_answer_token):
                        break     
                
        #answers.append(answer)
print float(count_contains)/total_question, count_similar/total_question, float(count_correct)/total_question
        
#         if ans == real_answer:
#             c += 1
#             else:
#                 if qa['answer'] in answer_list:
#                     print '*'*100
#                     print 'Question: ',text_question
#                     print 'lists: ',answer_list
#                     print 'scores: ',scores
#                     print 'pred: ',answer
#                     print 'actual: ',qa['answer']
#                     print 'sentence: ',sentences[answer_sentence_id]
# print c/total_question
# print total_count/total_question

processing:  0
processing:  1
processing:  2
processing:  3
processing:  4
0.845536609829 0.277259534503 0.205616850552


In [23]:
# 0.196975067943 sent 4 threshold 8
# 0.198511166253 t 5
# 0.202528654142 t 1
# 0.200519910197 t 3
# 0.202056008508 t 2
# 0.202056008508 sent = 3
# 0.202292331325 sent = 5
# 0.194021032731 sent = 1

# 0.202764976959
# 0.20028358738 for old scoring system

In [51]:
text = 'However, particularly intense near-IR light (e.g., from IR lasers, IR LED sources, or from bright daylight with the visible light removed by colored gels) can be detected up to approximately 780 nm, and will be perceived as red light.'
print get_continuous_chunks_sentence(text,0)

([('However', 'RB'), (',', ','), ('particularly', 'RB'), ('intense', 'JJ'), ('near-IR', 'JJ'), ('light', 'NN'), ('(', '('), ('e.g.', 'JJ'), (',', ','), ('from', 'IN'), ('IR', 'NNP'), ('lasers', 'NNS'), (',', ','), ('IR', 'NNP'), ('LED', 'NNP'), ('sources', 'NNS'), (',', ','), ('or', 'POSICC'), ('from', 'IN'), ('bright', 'JJ'), ('daylight', 'NN'), ('with', 'WITH'), ('the', 'DT'), ('visible', 'JJ'), ('light', 'NN'), ('removed', 'VBN'), ('by', 'IN'), ('colored', 'JJ'), ('gels', 'NNS'), (')', ')'), ('can', 'MD'), ('be', 'VB'), ('detected', 'VBN'), ('up', 'RP'), ('to', 'TO'), ('approximately', 'RB'), ('780', 'CD'), ('nm', 'NNS'), (',', ','), ('and', 'POSICC'), ('will', 'MD'), ('be', 'VB'), ('perceived', 'VBN'), ('as', 'IN'), ('red', 'JJ'), ('light', 'NN'), ('.', '.')], [(u'N', u'intense near-IR light'), (u'J', u'e.g.'), (u'N', u'IR lasers'), (u'N', u'IR LED sources'), (u'N', u'bright daylight'), (u'N', u'visible light'), (u'N', u'colored gels'), (u'N', u'780 nm'), (u'N', u'red light')])


In [24]:
lists = [u'year','y']
if u'y' in lists:
    print 'sss'

sss
